# Load data

In [1]:
reddit = spark.read.parquet("/var/reddit-parquet") # believe these are Subreddit Submissions, Comments (children of Submissions) do not seem to be included

In [2]:
type(reddit)

pyspark.sql.dataframe.DataFrame

In [3]:
len(reddit.columns)

70

In [4]:
reddit.printSchema()

root
 |-- _corrupt_record: string (nullable = true)
 |-- adserver_click_url: string (nullable = true)
 |-- adserver_imp_pixel: string (nullable = true)
 |-- approved_by: string (nullable = true)
 |-- archived: boolean (nullable = true)
 |-- author: string (nullable = true)
 |-- author_flair_css_class: string (nullable = true)
 |-- author_flair_text: string (nullable = true)
 |-- banned_by: string (nullable = true)
 |-- body: string (nullable = true)
 |-- body_html: string (nullable = true)
 |-- clicked: boolean (nullable = true)
 |-- controversiality: long (nullable = true)
 |-- created: long (nullable = true)
 |-- created_utc: string (nullable = true)
 |-- disable_comments: boolean (nullable = true)
 |-- distinguished: string (nullable = true)
 |-- domain: string (nullable = true)
 |-- downs: long (nullable = true)
 |-- edited: string (nullable = true)
 |-- from: string (nullable = true)
 |-- from_id: string (nullable = true)
 |-- from_kind: string (nullable = true)
 |-- gilded: long 

In [5]:
record_count = reddit.count()

In [6]:
record_count # 2859977347; 1% of this is +28 million

2859977347

> ### The DataFrame we created has a fairly large number of columns (70), is deeply nested in several instances (up to 8 layers deep), and contains a significant number of records (+2.8 billion).

In [7]:
# select a subset of columns for EDA

these_cols = [
    "id",
    "parent_id",
    "subreddit",
    "author",
    "created_utc",
    "body",
    "num_comments",
    "score"
] # "created" is not as populated as created_utc, better to use created_utc

> ### For our initial EDA we primarily care about the columns indicated above so we'll subset the data appropriately.

In [8]:
cols_df = reddit.select(these_cols)
cols_df.dtypes

[('id', 'string'),
 ('parent_id', 'string'),
 ('subreddit', 'string'),
 ('author', 'string'),
 ('created_utc', 'string'),
 ('body', 'string'),
 ('num_comments', 'bigint'),
 ('score', 'bigint')]

> ### It's worthwhile to check the data types of the columns. You may notice that "created_utc" is a string. It actually contains the created date in Unix time/Epoch time (the number of seconds since January 1, 1970). We'll want to convert that to a more human readable format.

# Created date

In [9]:
from pyspark.sql.functions import from_unixtime

In [10]:
# add new col showing just created year
cols_df = cols_df.withColumn(
    "created_utc_year", 
    from_unixtime(
        cols_df["created_utc"], 
        "yyyy" # full timestamp: yyyy-MM-dd HH:mm:ss.SS
    )
)

In [11]:
years_df = cols_df.groupby("created_utc_year").count()
years_df.orderBy("created_utc_year").show()

+----------------+---------+
|created_utc_year|    count|
+----------------+---------+
|            null|        1|
|            2005|     1086|
|            2006|   419341|
|            2007|  2745064|
|            2008|  9773673|
|            2009| 23726352|
|            2010| 55571522|
|            2011|138398080|
|            2012|289689090|
|            2013|441953216|
|            2014|584765776|
|            2015|738997386|
|            2016|573936760|
+----------------+---------+



> ### The data covers 11 years from 2005 to 2016. Reddit was founded in 2005 and we can see a hint of the platform's growth just in terms of the number of posts in the dataset.

In [12]:
null_count = cols_df[cols_df["body"].isNull()].count()

In [13]:
null_count

279383793

In [14]:
round((float(null_count)/record_count)*100,2)

9.77

> ### While there are a large number of records with null values in "body" (+279 million), this is only ~10% of the overall data.

In [15]:
clean_df = cols_df[
    (cols_df["created_utc_year"].isNotNull())&
    (cols_df["body"].isNotNull())
]

In [16]:
clean_df.count() # 2,580,593,554

2580593554

> ### We are still left with +2.5 billion records!
> ### However, using all the records would be excessive and time-intensive.
> ### One percent of the total, approximately 25.8 million, should be sufficient. 

In [17]:
these_years = years_df.select("created_utc_year").distinct()

In [18]:
yearly_portion = these_years.count()/100.0

In [19]:
fraction_dict = {str(i["created_utc_year"]):yearly_portion for i in these_years.collect()}

In [20]:
# stratified sample
one_perct_df = clean_df.sampleBy(
    col = "created_utc_year",
    fractions = fraction_dict
)

In [21]:
one_perct_df.count() # 334,312,335; 334 million?

335564606

In [22]:
one_perct_df.groupby("created_utc_year").count().orderBy("created_utc_year").show()

+----------------+--------+
|created_utc_year|   count|
+----------------+--------+
|            2005|     150|
|            2006|   54215|
|            2007|  321004|
|            2008|  941395|
|            2009| 2454977|
|            2010| 6306987|
|            2011|16041512|
|            2012|33862312|
|            2013|52311101|
|            2014|69152727|
|            2015|86917992|
|            2016|67200234|
+----------------+--------+



# Subreddits

In [23]:
from pyspark.sql.functions import round, datediff, from_utc_timestamp

In [24]:
# clean_df = clean_df.withColumn(
#     "created_utc_timestamp", 
#     from_unixtime(
#         clean_df["created_utc"], 
#         "yyyy-MM-dd HH:mm:ss.SS" 
#     )
# )

# clean_df = clean_df.withColumn(
#     "created_utc_int", 
#     from_utc_timestamp(
#         clean_df["created_utc"],
#         "EST"
#     )
# )

In [41]:
# # get range between min & max created_utc_year dates
# max_date = clean_df.agg({'created_utc': 'max'}).first()[0]
# min_date = clean_df.agg({'created_utc': 'min'}).first()[0]

In [42]:
# import datetime

In [43]:
# max_date_str = datetime.datetime.utcfromtimestamp(float(max_date)).strftime('%Y-%m-%dT%H:%M:%SZ')
# min_date_str = datetime.datetime.utcfromtimestamp(float(min_date)).strftime('%Y-%m-%dT%H:%M:%SZ')

In [44]:
# max_date, max_date_str

In [45]:
# min_date, min_date_str

### Need to find a way to get days between min & max date

In [46]:
# datetime.timedelta(seconds = int(max_date))

In [47]:
total_days = 3915 # calculated manually

In [49]:
subreddit_df = clean_df.groupby("subreddit").count()
subreddit_df = subreddit_df.withColumn("posts_per_day", round(subreddit_df["count"]/total_days,2))
subreddit_df.orderBy("count", ascending = False).show(10)

+---------------+---------+-------------+
|      subreddit|    count|posts_per_day|
+---------------+---------+-------------+
|      AskReddit|250715708|     64039.77|
|          funny| 59765702|     15265.82|
|           pics| 55969243|      14296.1|
|leagueoflegends| 43488861|     11108.27|
|         gaming| 38380071|      9803.34|
|       politics| 37038626|       9460.7|
|            WTF| 34459618|      8801.95|
|  AdviceAnimals| 32529745|       8309.0|
|         videos| 29746032|      7597.96|
|      worldnews| 29339958|      7494.24|
+---------------+---------+-------------+
only showing top 10 rows



> ### The Subreddit with the most records with non-null values in "body" is AskReddit which amounts to approximately 64,000 records per day.

> ### Is this representative of what's posted to Reddit?

> ### While I could not find official statistics from Reddit itself, I did find a 3rd party site that purports to track statistics on Subreddits. For instance, it shows that in a recent 24-hour period AskReddit received +12,000 Posts, funny received +1,300, worldnews received 657, and nfl received ~20.

> ### The AskReddit Subreddit features +28 million members.

In [34]:
# add new col showing year month day
cols_df = cols_df.withColumn(
    "created_utc_yearMonthDay", 
    from_unixtime(
        cols_df["created_utc"], 
        "yyyy-MM-dd" # full timestamp: yyyy-MM-dd HH:mm:ss.SS
    )
)

In [40]:
yearsMonthDay_df = cols_df.groupby(["subreddit","created_utc_yearMonthDay"]).count()
avg_df = yearsMonthDay_df.groupby("subreddit").avg()
avg_df.orderBy("avg(count)", ascending = False).show(10)

+---------------+------------------+
|      subreddit|        avg(count)|
+---------------+------------------+
|      AskReddit| 82573.53356665606|
|          funny|21205.329404268876|
|           pics| 19387.07508749602|
|leagueoflegends|       19302.51375|
|  AdviceAnimals|17288.425298329355|
|     The_Donald| 14156.92824074074|
|         gaming| 12662.88141809291|
|       politics|11853.600784550392|
|            WTF| 11560.75676536135|
|      thebutton| 11310.20588235294|
+---------------+------------------+
only showing top 10 rows



In [61]:
reddit.select("permalink").na.drop().sample(False, 0.1).show(3)

+--------------------+
|           permalink|
+--------------------+
|/r/atheism/commen...|
|/r/starcraft/comm...|
|/r/worldnews/comm...|
+--------------------+
only showing top 3 rows



# Can still evaluate distributions of scores & body_len for certain (common) Subreddits